In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb
import numpy as np
import pandas as pd

In [2]:
path = "./"
train = pd.read_csv(path + "input/train_set.csv")
test = pd.read_csv(path + "input/test_set.csv")
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 

In [3]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [4]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [5]:
train.corr() # 各特征之间的相关系数

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [6]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [7]:
test["y"] = -1
print(len(test.columns))

18


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         10852 non-null  int64 
 1   age        10852 non-null  int64 
 2   job        10852 non-null  object
 3   marital    10852 non-null  object
 4   education  10852 non-null  object
 5   default    10852 non-null  object
 6   balance    10852 non-null  int64 
 7   housing    10852 non-null  object
 8   loan       10852 non-null  object
 9   contact    10852 non-null  object
 10  day        10852 non-null  int64 
 11  month      10852 non-null  object
 12  duration   10852 non-null  int64 
 13  campaign   10852 non-null  int64 
 14  pdays      10852 non-null  int64 
 15  previous   10852 non-null  int64 
 16  poutcome   10852 non-null  object
 17  y          10852 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 1.5+ MB


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB


In [9]:
# reste_index 重置索引，原本的index变为一列数据，drop = True 不保留原来的index
data = train.append(test).reset_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [10]:
!pip install catboost==0.15.2 -i https://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [12]:
!pip install xgboost -i https://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [16]:
path = "./"
import os
import json
import math

# os.system("pip install numba")
from numba import jit

# os.system("pip install tqdm")
from tqdm import tqdm_notebook
from tqdm import tqdm

# 集成模型
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb

import numpy as np
import pandas as pd

# sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score

import time
import datetime
from datetime import datetime,timedelta

# 回收
import gc

# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist

from collections import Counter
from statistics import mode
from itertools import product
import ast

import warnings
warnings.filterwarnings("ignore")

In [18]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ["ID", "y"]]
cat_col

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [19]:
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data["count_" + i] = data.groupby([i])[i].transform("count")
    data[i] = lbl.fit_transform(data[i].astype(str))

In [20]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


1.ID Int客户唯一标识

2.age Int客户年龄

3.job String客户的职业

4.marital String婚姻状况

5.education String受教育水平

6.default String是否有违约记录

7.balance Int每年账户的平均余额

8.housing String是否有住房贷款

9.loan String是否有个人贷款

10.contact String与客户联系的沟通方式

11.day Int最后一次联系的时间（几号）

12.month String最后一次联系的时间（月份）

13.duration Int最后一次联系的交流时长

14.campaign Int在本次活动中，与该客户交流过的次数

15.pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17.poutcome String上一次活动的结果

18.y Int预测客户是否会订购定期存款业务

In [22]:
feats = [i for i in data.columns if i not in ["ID","y"]]
feats,len(feats)

(['age',
  'job',
  'marital',
  'education',
  'default',
  'balance',
  'housing',
  'loan',
  'contact',
  'day',
  'month',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'poutcome',
  'count_job',
  'count_marital',
  'count_education',
  'count_default',
  'count_housing',
  'count_loan',
  'count_contact',
  'count_month',
  'count_poutcome'],
 25)

In [25]:
model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    num_leaves=30,
    reg_alpha=0,
    reg_lambda=0,
    max_depth=-1,
    n_estimators=1500,
    objective="binary",
    metric="auc",
    subsample=0.95,
    colsample_bytree=0.7,
    subsample_freq=1,
    learning_rate=0.02,
    random_state=2022
)

# 因为test的“y”= -1
train_x =data[data["y"] != -1][feats]
train_y = data[data["y"] !=-1]["y"]
testx = data[data["y"] == -1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)

# 训练
model.fit(train_x, train_y)
# 预测
test_pre = model.predict_proba(testx)[:,1] # num_iteration = model.best_iteration

In [26]:
print(len(test_pre))

10852


In [27]:
pre = data[data["y"]==-1][["ID"]]
pre["pred"] = test_pre
print(len(pre))

10852


In [28]:
pre.head()

,ID,pred
25317,25318,0.025007
25318,25319,0.004888
25319,25320,0.000976
25320,25321,0.760124
25321,25322,0.017962


In [29]:
pre.to_csv("baseline.csv", index=False)

In [33]:
# 交叉验证
from sklearn.model_selection import KFold

n_splits = 10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
train_x = data[data["y"] != -1][feats]
train_y = data[data["y"] !=-1]["y"]
res = data[data["y"]==-1][["ID"]]

test_x = data[data["y"]==-1][feats]
res["pred"] = 0

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #(val_x, val_y)
    model.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)], eval_metric="auc", early_stopping_rounds=100)
    res["pred"] += model.predict_proba(test_x)[:,1]
    
res["pred"] = res["pred"] / 10
res.to_csv("./submission10_fold.csv", index=False)

[1]	training's auc: 0.893643	valid_1's auc: 0.888131
[2]	training's auc: 0.904946	valid_1's auc: 0.892659
[3]	training's auc: 0.908738	valid_1's auc: 0.898233
[4]	training's auc: 0.908876	valid_1's auc: 0.899155
[5]	training's auc: 0.909634	valid_1's auc: 0.899391
[6]	training's auc: 0.912386	valid_1's auc: 0.903799
[7]	training's auc: 0.914262	valid_1's auc: 0.905807
[8]	training's auc: 0.926337	valid_1's auc: 0.915087
[9]	training's auc: 0.926684	valid_1's auc: 0.915874
[10]	training's auc: 0.927633	valid_1's auc: 0.917027
[11]	training's auc: 0.926989	valid_1's auc: 0.91684
[12]	training's auc: 0.927234	valid_1's auc: 0.916579
[13]	training's auc: 0.927735	valid_1's auc: 0.917258
[14]	training's auc: 0.928684	valid_1's auc: 0.917474
[15]	training's auc: 0.929289	valid_1's auc: 0.918272
[16]	training's auc: 0.929539	valid_1's auc: 0.918322
[17]	training's auc: 0.929708	valid_1's auc: 0.91806
[18]	training's auc: 0.930036	valid_1's auc: 0.917716
[19]	training's auc: 0.929929	valid_1's

[170]	training's auc: 0.951605	valid_1's auc: 0.932256
[171]	training's auc: 0.95168	valid_1's auc: 0.932249
[172]	training's auc: 0.951783	valid_1's auc: 0.932275
[173]	training's auc: 0.951919	valid_1's auc: 0.932336
[174]	training's auc: 0.951986	valid_1's auc: 0.932386
[175]	training's auc: 0.952066	valid_1's auc: 0.93239
[176]	training's auc: 0.952174	valid_1's auc: 0.932415
[177]	training's auc: 0.952229	valid_1's auc: 0.932433
[178]	training's auc: 0.95231	valid_1's auc: 0.932489
[179]	training's auc: 0.952347	valid_1's auc: 0.932434
[180]	training's auc: 0.952418	valid_1's auc: 0.932433
[181]	training's auc: 0.952515	valid_1's auc: 0.932406
[182]	training's auc: 0.952598	valid_1's auc: 0.932356
[183]	training's auc: 0.952676	valid_1's auc: 0.932411
[184]	training's auc: 0.952727	valid_1's auc: 0.932486
[185]	training's auc: 0.952819	valid_1's auc: 0.932523
[186]	training's auc: 0.952871	valid_1's auc: 0.93244
[187]	training's auc: 0.952953	valid_1's auc: 0.932443
[188]	training

[337]	training's auc: 0.963679	valid_1's auc: 0.933344
[338]	training's auc: 0.96373	valid_1's auc: 0.933369
[339]	training's auc: 0.963785	valid_1's auc: 0.933369
[340]	training's auc: 0.963859	valid_1's auc: 0.933418
[341]	training's auc: 0.963936	valid_1's auc: 0.933469
[342]	training's auc: 0.963991	valid_1's auc: 0.93345
[343]	training's auc: 0.964044	valid_1's auc: 0.93344
[344]	training's auc: 0.964118	valid_1's auc: 0.933447
[345]	training's auc: 0.964189	valid_1's auc: 0.933425
[346]	training's auc: 0.964263	valid_1's auc: 0.933447
[347]	training's auc: 0.964308	valid_1's auc: 0.933449
[348]	training's auc: 0.964366	valid_1's auc: 0.933409
[349]	training's auc: 0.964457	valid_1's auc: 0.933415
[350]	training's auc: 0.964541	valid_1's auc: 0.933621
[351]	training's auc: 0.96462	valid_1's auc: 0.933664
[352]	training's auc: 0.964669	valid_1's auc: 0.933637
[353]	training's auc: 0.964725	valid_1's auc: 0.933658
[354]	training's auc: 0.964796	valid_1's auc: 0.933687
[355]	training

[515]	training's auc: 0.973004	valid_1's auc: 0.93524
[516]	training's auc: 0.973052	valid_1's auc: 0.935249
[517]	training's auc: 0.973084	valid_1's auc: 0.93523
[518]	training's auc: 0.973125	valid_1's auc: 0.935209
[519]	training's auc: 0.973159	valid_1's auc: 0.93522
[520]	training's auc: 0.973217	valid_1's auc: 0.93526
[521]	training's auc: 0.973259	valid_1's auc: 0.935271
[522]	training's auc: 0.973296	valid_1's auc: 0.935214
[523]	training's auc: 0.973328	valid_1's auc: 0.935234
[524]	training's auc: 0.973371	valid_1's auc: 0.935204
[525]	training's auc: 0.973422	valid_1's auc: 0.935161
[526]	training's auc: 0.973467	valid_1's auc: 0.935118
[527]	training's auc: 0.973507	valid_1's auc: 0.935133
[528]	training's auc: 0.973545	valid_1's auc: 0.935137
[529]	training's auc: 0.97359	valid_1's auc: 0.935174
[530]	training's auc: 0.973631	valid_1's auc: 0.935159
[531]	training's auc: 0.973661	valid_1's auc: 0.935152
[532]	training's auc: 0.973707	valid_1's auc: 0.935158
[533]	training'

[694]	training's auc: 0.979797	valid_1's auc: 0.935427
[695]	training's auc: 0.979834	valid_1's auc: 0.935442
[696]	training's auc: 0.979889	valid_1's auc: 0.935454
[697]	training's auc: 0.979917	valid_1's auc: 0.935485
[698]	training's auc: 0.979947	valid_1's auc: 0.935452
[699]	training's auc: 0.979988	valid_1's auc: 0.935466
[700]	training's auc: 0.980023	valid_1's auc: 0.935485
[701]	training's auc: 0.980078	valid_1's auc: 0.935457
[702]	training's auc: 0.980111	valid_1's auc: 0.935454
[703]	training's auc: 0.980153	valid_1's auc: 0.935482
[704]	training's auc: 0.980192	valid_1's auc: 0.935527
[705]	training's auc: 0.980223	valid_1's auc: 0.935519
[706]	training's auc: 0.980252	valid_1's auc: 0.935527
[707]	training's auc: 0.98028	valid_1's auc: 0.935538
[708]	training's auc: 0.980309	valid_1's auc: 0.935547
[709]	training's auc: 0.98033	valid_1's auc: 0.935552
[710]	training's auc: 0.980355	valid_1's auc: 0.93555
[711]	training's auc: 0.98038	valid_1's auc: 0.935555
[712]	training

...

[351]	training's auc: 0.964321	valid_1's auc: 0.93696
[352]	training's auc: 0.964365	valid_1's auc: 0.936928
[353]	training's auc: 0.96441	valid_1's auc: 0.936914
[354]	training's auc: 0.964502	valid_1's auc: 0.936964
[355]	training's auc: 0.964559	valid_1's auc: 0.937063
[356]	training's auc: 0.964635	valid_1's auc: 0.937022
[357]	training's auc: 0.96469	valid_1's auc: 0.936981
[358]	training's auc: 0.964744	valid_1's auc: 0.936969
[359]	training's auc: 0.964802	valid_1's auc: 0.936998
[360]	training's auc: 0.96488	valid_1's auc: 0.937058
[361]	training's auc: 0.96495	valid_1's auc: 0.937084
[362]	training's auc: 0.965018	valid_1's auc: 0.937114
[363]	training's auc: 0.965059	valid_1's auc: 0.937121
[364]	training's auc: 0.965113	valid_1's auc: 0.937138
[365]	training's auc: 0.965191	valid_1's auc: 0.937102
[366]	training's auc: 0.965242	valid_1's auc: 0.937155
[367]	training's auc: 0.965287	valid_1's auc: 0.937172
[368]	training's auc: 0.965357	valid_1's auc: 0.937197
[369]	training'

[504]	training's auc: 0.972002	valid_1's auc: 0.937919
[505]	training's auc: 0.972036	valid_1's auc: 0.937907
[506]	training's auc: 0.972069	valid_1's auc: 0.937928
[507]	training's auc: 0.972097	valid_1's auc: 0.937881
[508]	training's auc: 0.972142	valid_1's auc: 0.93788
[509]	training's auc: 0.97221	valid_1's auc: 0.937851
[510]	training's auc: 0.972249	valid_1's auc: 0.93787
[511]	training's auc: 0.972277	valid_1's auc: 0.937851
[512]	training's auc: 0.972307	valid_1's auc: 0.937828
[513]	training's auc: 0.972351	valid_1's auc: 0.937821
[514]	training's auc: 0.972378	valid_1's auc: 0.937828
[515]	training's auc: 0.972397	valid_1's auc: 0.937838
[516]	training's auc: 0.972429	valid_1's auc: 0.93786
[517]	training's auc: 0.972475	valid_1's auc: 0.937911
[518]	training's auc: 0.972503	valid_1's auc: 0.937887
[519]	training's auc: 0.972545	valid_1's auc: 0.937886
[520]	training's auc: 0.972583	valid_1's auc: 0.937866
[521]	training's auc: 0.972608	valid_1's auc: 0.937887
[522]	training

[686]	training's auc: 0.979232	valid_1's auc: 0.938313
[687]	training's auc: 0.979274	valid_1's auc: 0.938313
[688]	training's auc: 0.979318	valid_1's auc: 0.93833
[689]	training's auc: 0.979351	valid_1's auc: 0.938338
[690]	training's auc: 0.979396	valid_1's auc: 0.938324
[691]	training's auc: 0.979423	valid_1's auc: 0.938331
[692]	training's auc: 0.979459	valid_1's auc: 0.938338
[693]	training's auc: 0.979486	valid_1's auc: 0.938324
[694]	training's auc: 0.979519	valid_1's auc: 0.938302
[695]	training's auc: 0.979554	valid_1's auc: 0.938274
[696]	training's auc: 0.97958	valid_1's auc: 0.938266
[697]	training's auc: 0.979608	valid_1's auc: 0.938278
[698]	training's auc: 0.97963	valid_1's auc: 0.938287
[699]	training's auc: 0.979659	valid_1's auc: 0.938291
[700]	training's auc: 0.979682	valid_1's auc: 0.938326
[701]	training's auc: 0.979713	valid_1's auc: 0.938313
[702]	training's auc: 0.979743	valid_1's auc: 0.938315
[703]	training's auc: 0.979764	valid_1's auc: 0.938299
[704]	trainin

[87]	training's auc: 0.94471	valid_1's auc: 0.919913
[88]	training's auc: 0.944839	valid_1's auc: 0.919877
[89]	training's auc: 0.944924	valid_1's auc: 0.919919
[90]	training's auc: 0.945044	valid_1's auc: 0.91991
[91]	training's auc: 0.945169	valid_1's auc: 0.919782
[92]	training's auc: 0.945394	valid_1's auc: 0.919993
[93]	training's auc: 0.945494	valid_1's auc: 0.920071
[94]	training's auc: 0.945631	valid_1's auc: 0.920115
[95]	training's auc: 0.945736	valid_1's auc: 0.920186
[96]	training's auc: 0.945813	valid_1's auc: 0.920106
[97]	training's auc: 0.945903	valid_1's auc: 0.920163
[98]	training's auc: 0.945952	valid_1's auc: 0.920139
[99]	training's auc: 0.946121	valid_1's auc: 0.920145
[100]	training's auc: 0.946245	valid_1's auc: 0.920097
[101]	training's auc: 0.946382	valid_1's auc: 0.920087
[102]	training's auc: 0.946412	valid_1's auc: 0.920014
[103]	training's auc: 0.946574	valid_1's auc: 0.920277
[104]	training's auc: 0.946685	valid_1's auc: 0.920295
[105]	training's auc: 0.9

[266]	training's auc: 0.960104	valid_1's auc: 0.923016
[267]	training's auc: 0.960145	valid_1's auc: 0.922974
[268]	training's auc: 0.960218	valid_1's auc: 0.922965
[269]	training's auc: 0.960288	valid_1's auc: 0.922985
[270]	training's auc: 0.96036	valid_1's auc: 0.923031
